In [1]:
# Function to create hierarchical structured data in .json format for regions, countries and ingredient categories.
# Authors: Mehrin Azan, Clarissa Auckenthaler



import numpy as np
import pandas as pd
import json

In [2]:
df= pd.read_csv("https://raw.githubusercontent.com/SirMaik/Visualizing-Recipes/main/static/data/category_counts.csv")
df['name']= "recipes"

In [4]:
df= df.set_index(['name','region','country']).reset_index()
x2= df.melt(id_vars=['name','region','country'], 
        var_name="ingredient_categorie", 
        value_name="counts")

In [5]:
x2 = x2.sort_values(by = ['country', 'region'], ascending = [True, False], na_position = 'first')
x2 = x2.drop(columns=['name'])


,name,region,country,ingredient_categorie,counts
60,recipes,north-american,american,vegetable,28272
129,recipes,north-american,american,dish,3779
198,recipes,north-american,american,mix,2093
267,recipes,north-american,american,dairy,32653
336,recipes,north-american,american,sauce,9269
...,...,...,...,...,...
1773,recipes,european,welsh,sweetener,101
1842,recipes,european,welsh,seed,2
1911,recipes,european,welsh,flavoring,10
1980,recipes,european,welsh,fungi,1


In [7]:
df=x2.copy()

# initialize a flare dictionary
flare = {"name": "recipes", "children": []}

# iterate through dataframe values
for row in df.values:
    level0 = row[0]
    level1 = row[1]
    level2 = row[2]
    value = row[3]
    
    # create a dictionary with all the row data
    d = {'name': level0,
          'children': [{'name': level1,
                        'children': [{'name': level2,
                                      'value': value}]}]}
    # initialize key lists
    key0 = []
    key1 = []

    # iterate through first level node names
    for i in flare ['children']:
        key0.append(i['name'])

        # iterate through next level node names
        key1 = []
        for _, v in i.items():
            if isinstance(v, list):
                for x in v:
                    key1.append(x['name'])

    # add the full row of data if the root is not in key0
    if level0 not in key0:
        d = {'name': level0,
              'children': [{'name': level1,
                            'children': [{'name': level2,
                                          'value': value}]}]}
        flare['children'].append(d)

    elif level1 not in key1:

        # if the root exists, then append only the next level children

        d = {'name': level1,
              'children': [{'name': level2,
                            'value': value}]}

        flare['children'][key0.index(level0)]['children'].append(d)

    else:

        # if the root exists, then only append the next level children
        
        d = {'name': level2,
             'value': value}

        flare['children'][key0.index(level0)]['children'][key1.index(level1)]['children'].append(d)

# uncomment next three lines to save as json file
# save to some file
# with open('filename_here.json', 'w') as outfile:
#     json.dump(flare, outfile)

print(json.dumps(flare, indent=2))

{
  "name": "recipes",
  "children": [
    {
      "name": "recipes",
      "children": [
        {
          "name": "north-american",
          "children": [
            {
              "name": "american",
              "value": "vegetable"
            },
            {
              "name": "american",
              "value": "dish"
            },
            {
              "name": "american",
              "value": "mix"
            },
            {
              "name": "american",
              "value": "dairy"
            },
            {
              "name": "american",
              "value": "sauce"
            },
            {
              "name": "american",
              "value": "herb"
            },
            {
              "name": "american",
              "value": "confectionery"
            },
            {
              "name": "american",
              "value": "spread"
            },
            {
              "name": "american",
              "value": "meat"
 

In [53]:
# Export json file
with open('filename_here.json', 'w') as outfile:
     json.dump(flare, outfile)